In [6]:
%%spark 

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [5]:
df_song = spark.read.json("s3a://udacity-dend/song_data/*/*/*")

VBox()

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [6]:
df_log = spark.read.json("s3a://udacity-dend/log_data/*/*/*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
 df_log.show(n=2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|The Big Gundown|

In [118]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import  IntegerType,StringType,DecimalType,LongType,DoubleType

song_schema = StructType([
    StructField('artist_id',StringType()),  
    StructField('artist_latitude',DoubleType()),
    StructField('artist_location',StringType()),
    StructField('artist_longitude',DoubleType()),
    StructField('artist_name',StringType()),
    StructField('duration',DoubleType()),
    StructField('num_songs' ,IntegerType()),
    StructField('song_id',StringType()),
    StructField('title',StringType()),
    StructField('year',IntegerType())   
])
log_schema = StructType([
    StructField('artist',StringType()),
    StructField('auth',StringType()),
    StructField('firstName',StringType()),
    StructField('gender',StringType()),
    StructField('itemInSession',IntegerType()),
    StructField('lastName',StringType()),
    StructField('length',DoubleType()),
    StructField('level',StringType()),
    StructField('location',StringType()),
    StructField('method',StringType()),
    StructField('page',StringType()),
    StructField('registration',StringType()),
    StructField('sessionId',IntegerType()),    
    StructField('song',StringType()),
    StructField('status',IntegerType()),
    StructField('ts',LongType()),
    StructField('userAgent',StringType()),
    StructField('userId',StringType())
    
])
df_song = spark.read.json("s3a://udacity-dend/song_data/*/*/*",schema=song_schema)
df_log = spark.read.json("s3a://udacity-dend/log_data/*/*/*.json",schema=log_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import to_timestamp
dim_user_columns = ['userid','firstname','lastName','gender','level']
dim_song_columns = ['song_id','artist_id','duration','title','year']
dim_user = df_log.selectExpr(*dim_user_columns).dropDuplicates().filter(col('userid').isNotNull()) 
dim_song = df_song.selectExpr(*dim_song_columns).dropDuplicates() 
df_log.selectExpr("cast(ts/1000 as timestamp) as time").alias('time').createOrReplaceTempView("time_table")
dim_time = spark.sql("select DISTINCT time, hour(time) as hour, day(time) as day,weekofyear(time) as week,\
          month(time) as month,year(time) as year,date_format(time, 'EEEE') as weekday  FROM time_table")
dim_artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
dim_artist =  df_song.selectExpr(*dim_artist_columns).dropDuplicates()

#####################################
dim_user.show(n=5)
dim_song.show(n=5)
dim_time.show(n=5)
dim_artist.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userid|firstname|lastName|gender|level|
+------+---------+--------+------+-----+
|    26|     Ryan|   Smith|     M| free|
|     7|   Adelyn|  Jordan|     F| free|
|    71|   Ayleen|    Wise|     F| free|
|    81|   Sienna|   Colon|     F| free|
|    87|   Dustin|     Lee|     M| free|
+------+---------+--------+------+-----+
only showing top 5 rows

+------------------+------------------+---------+--------------------+----+
|           song_id|         artist_id| duration|               title|year|
+------------------+------------------+---------+--------------------+----+
|SOUJLXS12A3F1ECF59|ARG9CID1187B9AB010|168.25424|Catholic Knees (A...|2009|
|SOAYZDX12A8C13B9C1|ARNUFGE1187B9B7881|  138.762|Did You Ever Go S...|1963|
|SOAXYPG12A8C1403DD|ARVJR1N1187FB4144A|226.48118|Set To Fail (Albu...|2009|
|SOEPHJQ12AB0184C9F|AR3VK5C1187B989C63|280.05832|Empire Falls (alb...|2003|
|SOZPIAY12A58A7B7AD|ARI18NF1187B99FC57|233.53424|Word Of Mouth (Al...|1981

In [178]:
 from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.window import Window
from pyspark.sql import functions as F 

df_songplay = df_log.where(col('page') == 'NextSong').join(dim_song,df_log.song  == dim_song.title) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
df_songplay.cache()
df_songplay.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [ ]:
df_songplay.columns

In [ ]:
df_log = spark.read.json("s3a://udacity-dend/log_data/*/*/*")

In [54]:
#df_log.where(col('ts').isNotNull()).select('ts').show(n=6)
df_log1.where(col('ts').isNotNull()).select('ts').show(n=6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|           ts|
+-------------+
|1542241826796|
|1542242481796|
|1542242741796|
|1542247071796|
|1542252577796|
|1542253449796|
+-------------+
only showing top 6 rows

In [109]:
 df_song1 = spark.read.json("s3a://udacity-dend/song_data/*/*/*").limit(5000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
df_song1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [50]:
df_log1.select('ts').show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|           ts|
+-------------+
|1542241826796|
|1542242481796|
|1542242741796|
|1542247071796|
|1542252577796|
+-------------+
only showing top 5 rows

In [77]:
#df_log.withColumn('time',to_timestamp(cast(df_log['ts']/1000 , 'int')).show(n=5)
df_log.selectExpr("cast( ts/1000 as timestamp)").show(n=5)

#df2 = df1.withColumn("TimestampType",F.to_timestamp(df1["unix_timestamp"]))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+
|CAST((ts / 1000) AS TIMESTAMP)|
+------------------------------+
|          2018-11-15 00:30:...|
|          2018-11-15 00:41:...|
|          2018-11-15 00:45:...|
|          2018-11-15 01:57:...|
|          2018-11-15 03:29:...|
+------------------------------+
only showing top 5 rows